In [1]:
!python -V

Python 3.5.6 :: Anaconda, Inc.


In [ ]:
!pip install kfp --upgrade

In [31]:
import kfp
from kfp import gcp
from kfp import dsl

import os
from datetime import datetime
import kfp.compiler as compiler

In [32]:
!pip freeze | grep kfp

kfp==1.0.0
kfp-server-api==1.0.0


In [33]:
client = kfp.Client(host='6fbf91bbac646e9a-dot-us-central2.pipelines.googleusercontent.com')

In [34]:
# Set your output and project
YOUR_NAME = 'erwinh' #change to your name
EXPERIMENT_NAME = 'Scikit Learn V1.15 %s' % (YOUR_NAME) # change to your experiment name
PROJECT_NAME =  'erwinh-ml-demo' # change this to your project name

In [35]:
experiment = client.create_experiment(EXPERIMENT_NAME)

In [36]:
#TODO change to KFP github
component_url_prefix = 'https://raw.githubusercontent.com/Huize501/kfp/master/scikit-component/'

In [37]:
dummy_op = kfp.components.load_component_from_url(component_url_prefix + 'component.yaml') 

In [41]:
@kfp.dsl.pipeline(name='KFP Scikit-learn', description='sklearn pipeline')
def my_pipeline():
    
    dummy1_task = dummy_op(
        # Input name "Input 1 URI" is converted to pythonic parameter name "input_1_uri"
        pathdata ='gs://erwinh-public-data/scikit/data/marketing-data.csv', 
        storage ='GCS',
        pathoutput ='gs://erwinh-ml-demos/output-scikit/',
        bqtable = 'kfp-primer-workshop.marketing_data.raw',
        ).apply(kfp.gcp.use_gcp_secret('user-gcp-sa'))

In [42]:
pipeline_func = my_pipeline  
pipeline_filename = pipeline_func.__name__ + '.pipeline.zip'
compiler.Compiler().compile(pipeline_func, pipeline_filename)

In [43]:
#Specify pipeline argument values
arguments = {}

#Submit a pipeline run
run_name = pipeline_func.__name__ + ' run %s' %datetime.now()
run_result = client.run_pipeline(experiment.id, run_name, pipeline_filename, arguments)

In [ ]:
# Copyright 2018 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.